In [1]:
from __future__ import division

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import pyomo.environ as pyo, numpy as np, pandas as pd
from pyomo.environ import *
from pyomo.opt import SolverFactory


geradores = pd.read_excel('input.xlsx', sheet_name='geradores')
demanda   = pd.read_excel('input.xlsx', sheet_name='demanda')


In [2]:
demanda

,t,dem
0,0,355
1,1,480
2,2,650
3,3,758
4,4,791
5,5,885
6,6,899
7,7,1005
8,8,955
9,9,814


In [3]:
# Entrada de dados:
#
# Geradores
cos = geradores.Custo.values
a = geradores.A.values*cos
b = geradores.B.values*cos
c = geradores.C.values*cos
pmin = geradores.Pmin.values
pmax = geradores.Pmax.values
ur = geradores.UR.values
dr = geradores.DR.values
#cos = geradores.Custo.to_list()
#pmin = geradores.Pmin.to_list()
#pmax = geradores.Pmax.to_list()
#ur = geradores.UR.to_list()
#dr = geradores.DR.to_list()

# Demanda
Pd = demanda.dem.values
t = demanda.t.to_list()

Ng = len(geradores)
Nd = len(Pd)
c

array([0.001278, 0.00194 , 0.00482 ])

In [4]:
model = pyo.ConcreteModel()

model.Pg = pyo.Var(range(Ng), range(Nd), bounds=(0,None))
Pg = model.Pg
model.deficit = pyo.Var(range(Nd), bounds=(0,None))
deficit = model.deficit

In [5]:
#objetivo

model.obj = pyo.Objective(expr= sum(sum([a[g] + b[g]*Pg[g,t] + c[g]*Pg[g,t]*Pg[g,t] for g in range(Ng)]) + deficit[t]*99999 for t in range(Nd)), sense=minimize)

In [6]:
# balanco:
model.balanco = ConstraintList()
for t in range(Nd):
    sum_Pg = sum(Pg[g,t] for g in range(Ng))
    model.balanco.add(expr= (sum_Pg + deficit[t]) == Pd[t])

In [7]:
# limites geração:
model.limger = pyo.ConstraintList()
for g in range(Ng):
    for t in range(Nd):
        model.limger.add(inequality(pmin[g], Pg[g,t], pmax[g]))


In [8]:
# limites de rampas
model.ramp_up = pyo.ConstraintList()
for g in range(Ng):
    for t in range(Nd):
        if(t==0):
            Constraint.Skip
        else:
            model.ramp_up.add(expr= Pg[g,t] - Pg[g,t-1] <= ur[g])

model.ramp_down = pyo.ConstraintList()
for g in range(Ng):
    for t in range(Nd):
        if(t==0):
            Constraint.Skip
        else:
            model.ramp_down.add(expr= Pg[g,t-1] - Pg[g,t] <= dr[g])

In [9]:
opt = SolverFactory('couenne', executable='C:\\couenne\\bin\\couenne.exe')
opt.solve(model)

model.pprint()

8 Set Declarations
    Pg_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain                : Size : Members
        None :     2 : Pg_index_0*Pg_index_1 :   72 : {(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8), (0, 9), (0, 10), (0, 11), (0, 12), (0, 13), (0, 14), (0, 15), (0, 16), (0, 17), (0, 18), (0, 19), (0, 20), (0, 21), (0, 22), (0, 23), (1, 0), (1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (1, 8), (1, 9), (1, 10), (1, 11), (1, 12), (1, 13), (1, 14), (1, 15), (1, 16), (1, 17), (1, 18), (1, 19), (1, 20), (1, 21), (1, 22), (1, 23), (2, 0), (2, 1), (2, 2), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (2, 8), (2, 9), (2, 10), (2, 11), (2, 12), (2, 13), (2, 14), (2, 15), (2, 16), (2, 17), (2, 18), (2, 19), (2, 20), (2, 21), (2, 22), (2, 23)}
    Pg_index_0 : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {0, 1, 2}
    Pg_index_1 : Size=1, Index=None, Ordered=False

In [10]:
#for g in range(Ng):
 #   for t in range(Nd):
        #print(f"Pg[{g},{t}] = {pyo.value(Pg[g,t])} / Pd[{t}] == {pyo.value(Pd[t])} / Corte: {pyo.value(deficit[t])}")



for t in range(Nd):
    print(f'Pd[{t}] = {pyo.value(Pd[t])} MW')
    for g in range(Ng):
        print(f' Pg[{g}] = {pyo.value(Pg[g,t])}')
    
    print(f'Pt[{t}] = {sum(pyo.value(Pg[:,t]))}')
    print(f'corte[{t}] = {pyo.value(deficit[t])}\n')

        

Pd[0] = 355 MW
 Pg[0] = 205.0
 Pg[1] = 100.0
 Pg[2] = 50.0
Pt[0] = 355.0
corte[0] = 0.0

Pd[1] = 480 MW
 Pg[0] = 330.0
 Pg[1] = 100.0
 Pg[2] = 50.0
Pt[1] = 480.0
corte[1] = 0.0

Pd[2] = 650 MW
 Pg[0] = 500.0
 Pg[1] = 100.0
 Pg[2] = 50.0
Pt[2] = 650.0
corte[2] = 0.0

Pd[3] = 758 MW
 Pg[0] = 600.0
 Pg[1] = 107.99999999999997
 Pg[2] = 50.00000000000001
Pt[3] = 758.0
corte[3] = 0.0

Pd[4] = 791 MW
 Pg[0] = 600.0
 Pg[1] = 141.00000000000003
 Pg[2] = 50.00000000000001
Pt[4] = 791.0
corte[4] = 0.0

Pd[5] = 885 MW
 Pg[0] = 600.0
 Pg[1] = 212.08700815383474
 Pg[2] = 72.9129918461653
Pt[5] = 885.0
corte[5] = 0.0

Pd[6] = 899 MW
 Pg[0] = 595.9999999999998
 Pg[1] = 231.00000000000026
 Pg[2] = 71.99999999999994
Pt[6] = 899.0
corte[6] = 0.0

Pd[7] = 1005 MW
 Pg[0] = 577.0
 Pg[1] = 311.0000000000003
 Pg[2] = 116.99999999999994
Pt[7] = 1005.0000000000002
corte[7] = 0.0

Pd[8] = 955 MW
 Pg[0] = 532.0
 Pg[1] = 320.0
 Pg[2] = 101.99999999999994
Pt[8] = 954.0
corte[8] = 1.0000000000000249

Pd[9] = 814 MW


In [24]:
#We write some of the results in a csv file
f = open('gen_result.csv', 'w')

f.write("t,Pd,")
for g in range(Ng):
    f.write(f"G[{g+1}],")
f.write("Gerado,Corte"+"\n")

for t in range(Nd):
    f.write(str(t)+","+str(Pd[t])+",")
    for g in range(Ng):
        f.write(str(pyo.value(Pg[g,t]))+",")
    f.write(str(sum(pyo.value(Pg[:,t])))+","+str(pyo.value(deficit[t]))+"\n")

'''
for g in instance.g.value:
  for t in instance.t.value:
    f.write(str(g)+","+str(t)+","+str(instance.gen[g,t].value)+"\n")
'''


'\nfor g in instance.g.value:\n  for t in instance.t.value:\n    f.write(str(g)+","+str(t)+","+str(instance.gen[g,t].value)+"\n")\n'